In [1]:
import time, os, csv
import json, re, random
from requests import session
from lxml.html import fromstring
from fake_useragent import UserAgent
from DrissionPage import ChromiumOptions
from DrissionPage.common import Keys
from DrissionPage import Chromium, ChromiumPage
ua = UserAgent()

In [10]:
browser = ChromiumPage()
time.sleep(2)

option = ChromiumOptions()
driver = browser.new_tab()


# option.headless()

In [ ]:

page_url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco%2C+CA&start=0'


In [ ]:
soup = fromstring(driver.html)
nextPage = '//div[@role="navigation"]//span[text() = "Next Page"]'

In [5]:
contaier = soup.xpath('//div[@data-testid="serp-ia-card"]//div[contains(@class,"businessName")]//a')

In [1]:
from urllib.parse import urlencode
page = 0
params = {
        "find_desc": "Restaurants",
        "find_loc": "San Francisco, CA",
        "start": page,
        "dd_referrer": ""
    }
page_url = "https://www.yelp.com/search?" + urlencode(params)


print(page_url)

https://www.yelp.com/search?find_desc=Restaurants&find_loc=San+Francisco%2C+CA&start=0&dd_referrer=


In [ ]:
# business site 
websiteTag = soup.xpath('//div[@class="y-css-8x4us"]//p[text()="Business website"]//following-sibling::p/a')
website_url = websiteTag[0].get('href').strip() if websiteTag else 'site not found'
print(website_url)
addressTag = soup.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
address = addressTag[0].text_content().strip() if addressTag else 'site not found'
print(address)

site not found


In [9]:
'//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p'

addressTag = soup.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
address = addressTag[0].text_content().strip() if addressTag else 'site not found'
print(address)

6314 Geary Blvd San Francisco, CA 94121


In [ ]:
url = 'https://www.drissionpage.cn/'

driver.get(url)
time.sleep(3)

scroll_page = driver.ele('//body')


In [ ]:
def read_urls(f):
    with open(f,'r',encoding='utf-8') as x:return [i.strip() for i in x if i.strip()]

def scrape_yelp_page(p,u):
    try:
        p.get(u);p._wait_loaded(3)
        s=fromstring(p.html)
        t=s.xpath('//div[contains(@class,"headingLight")]/h1')
        n=t[0].text.strip() if t else 'N/A'
        ph=s.xpath('//div[@class="y-css-8x4us"]//p[text()="Phone number"]/following-sibling::p')
        phn=ph[0].text.strip() if ph else 'N/A'
        w=s.xpath('//div[@class="y-css-8x4us"]//p[text()="Business website"]/following-sibling::p/a')
        ws=w[0].get('href').strip() if w else 'N/A'
        a=s.xpath('//div[@class="y-css-8x4us"]//a[text()="Get Directions"]/../following-sibling::p')
        ad=a[0].text_content().strip() if a else 'N/A'
        return {'Business Name':n,'Phone Number':phn,'Website':ws,'Address':ad,'URL':u}
    except Exception as e:
        print(f"Error scraping {u}:{e}")
        return {'Business Name':'Error','Phone Number':'Error','Website':'Error','Address':'Error','URL':u}

def save_to_csv(d,f='yelp_data.csv'):
    if not d:return
    k=d[0].keys()
    with open(f,'w',newline='',encoding='utf-8') as x:
        w=csv.DictWriter(x,fieldnames=k)
        w.writeheader();w.writerows(d)
    print(f"Saved to {f}")